In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [3]:
df = pd.read_csv('data/ckd_clean.csv')
df

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,...,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia
0,48.0,80.0,1.020,1.0,0.0,0.166667,0.0,0.0,0.0,121.000000,...,15.4,44.0,7800.0,5.200000,1.0,1.0,0.0,0.0,0.0,0.0
1,7.0,50.0,1.020,4.0,0.0,0.083333,0.0,0.0,0.0,110.958333,...,11.3,38.0,6000.0,5.329167,0.0,0.0,0.0,0.0,0.0,0.0
2,62.0,80.0,1.010,2.0,3.0,0.000000,0.0,0.0,0.0,423.000000,...,9.6,31.0,7500.0,3.987500,0.0,1.0,0.0,1.0,0.0,1.0
3,48.0,70.0,1.005,4.0,0.0,0.000000,1.0,1.0,0.0,117.000000,...,11.2,32.0,6700.0,3.900000,1.0,0.0,0.0,1.0,1.0,1.0
4,51.0,80.0,1.010,2.0,0.0,0.000000,0.0,0.0,0.0,106.000000,...,11.6,35.0,7300.0,4.600000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,0.000000,0.0,0.0,0.0,140.000000,...,15.7,47.0,6700.0,4.900000,0.0,0.0,0.0,0.0,0.0,0.0
396,42.0,70.0,1.025,0.0,0.0,0.000000,0.0,0.0,0.0,75.000000,...,16.5,54.0,7800.0,6.200000,0.0,0.0,0.0,0.0,0.0,0.0
397,12.0,80.0,1.020,0.0,0.0,0.000000,0.0,0.0,0.0,100.000000,...,15.8,49.0,6600.0,5.400000,0.0,0.0,0.0,0.0,0.0,0.0
398,17.0,60.0,1.025,0.0,0.0,0.000000,0.0,0.0,0.0,114.000000,...,14.2,51.0,7200.0,5.900000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load your dataset
# Replace 'your_dataset.csv' with the path to your dataset
data = pd.read_csv('data/ckd_clean.csv')

# Preprocess the data
# Replace 'target_column' with the name of your target column
X = data.drop(columns=['class'])
y = data['class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Feature importance (optional)
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importances)

Accuracy: 0.9763157894736842
Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       142
           1       1.00      0.96      0.98       238

    accuracy                           0.98       380
   macro avg       0.97      0.98      0.98       380
weighted avg       0.98      0.98      0.98       380

Feature Importances:
                     Feature  Importance
2           specific_gravity    0.178324
11          serum_creatinine    0.131896
14               haemoglobin    0.090235
3                    albumin    0.086581
15        packed_cell_volume    0.084102
17      red_blood_cell_count    0.077734
9       blood_glucose_random    0.065872
18              hypertension    0.053095
12                    sodium    0.044131
19         diabetes_mellitus    0.042756
16    white_blood_cell_count    0.032148
0                        age    0.030434
1             blood_pressure    0.019540
6          abnormal_pus_ce

In [68]:
import numpy as np
import pandas as pd
from collections import Counter

# Define a Decision Tree Classifier
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.features = X.columns
        self.tree = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X.to_numpy()])

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes)]
        predicted_class = np.argmax(num_samples_per_class)
        node = {"predicted_class": predicted_class}

        if depth < self.max_depth:
            idx, threshold = self._best_split(X, y)
            if idx is not None:
                indices_left = X.iloc[:, idx] < threshold
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node["feature_index"] = idx
                node["threshold"] = threshold
                node["left"] = self._grow_tree(X_left, y_left, depth + 1)
                node["right"] = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _best_split(self, X, y):
        best_idx, best_threshold = None, None
        best_gini = 1.0
        for idx in range(X.shape[1]):
            thresholds, classes = zip(*sorted(zip(X.iloc[:, idx], y)))
            num_left = [0] * self.n_classes
            num_right = Counter(classes)
            for i in range(1, len(y)):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes))
                gini_right = 1.0 - sum((num_right[x] / (len(y) - i)) ** 2 for x in range(self.n_classes))
                gini = (i * gini_left + (len(y) - i) * gini_right) / len(y)
                if thresholds[i] != thresholds[i - 1] and gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_threshold = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_threshold

    def _predict(self, inputs):
        node = self.tree
        while "feature_index" in node:
            if inputs[node["feature_index"]] < node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["predicted_class"]

# Define the Random Forest Classifier
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=None, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(X), len(X), replace=True)
            X_bootstrap = X.iloc[bootstrap_indices]
            y_bootstrap = y.iloc[bootstrap_indices]
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return [Counter(tree_preds[:, i]).most_common(1)[0][0] for i in range(X.shape[0])]

# Example usage
# Load your dataset
# Replace 'your_dataset.csv' and 'target_column' with your actual dataset and target column
data = pd.read_csv('data/ckd_clean.csv')
# Split the data into training and testing sets
def train_test_split(X, y, test_size=0.5, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

# Perform the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Train the Random Forest on the training set
rf = RandomForest(n_estimators=100, max_depth=5)
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate accuracy on the test set
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.98
